# Torch MC PILCO

In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [4]:
import gpytorch
import torch
import numpy as np
import gymnasium as gym

from torchrl.collectors import SyncDataCollector
from tensordict import TensorDict
from torchrl.envs.libs.gym import GymEnv
from torchrl.envs.utils import RandomPolicy
from torchrl.data import ReplayBuffer
from torchrl.data import LazyTensorStorage
from torch.func import vmap, functional_call

from torch_pilco.model_learning.dynamical_models import (
    DynamicalModel,
    fit,
    data_to_gp_input,
    data_to_policy_input
)
from torch_pilco.policy_learning.controllers import RandomExploration

In [5]:
import functools
from torch.func import vmap, functional_call

In [6]:
import matplotlib as mpl
import matplotlib.pyplot as plt

cols = mpl.rcParams["axes.prop_cycle"].by_key()["color"]
%matplotlib inline

## Functions

In [8]:
def build_pendulum_training_data(
    data_tensordict: TensorDict,
 ) -> tuple[torch.Tensor, torch.Tensor]:
    return data_tensordict['observation'].float(), data_tensordict['action'].float()

## Global Parameters

In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
frames_per_batch = 100

env = GymEnv("Pendulum-v1")
random_policy = RandomPolicy(env.action_spec)

In [10]:
position_memory: int = 2
control_memory: int = 1
queue_size = 1 + max(control_memory, position_memory)
num_particles = 400

In [11]:
data_to_policy_input_closure = functools.partial(data_to_policy_input, position_memory=position_memory)

In [12]:
data_to_gp_input_closure = functools.partial(data_to_gp_input, control_memory=control_memory, position_memory=position_memory)

## Generate initial experimenal data from environment

In [13]:
# Generate a random trajectory from the environment
collector = SyncDataCollector(
    env,
    policy=random_policy,
    frames_per_batch=frames_per_batch,
    total_frames=frames_per_batch,
)
# Now determine how many frames are stacked for the dynamical model input:

replay_buffer = ReplayBuffer(storage=LazyTensorStorage(10000))

## Main Loop

In [14]:
# %load src/torch_pilco/pendulum.py
for data in collector:
    # convert the tensordict from collector to a version
    # suitable for dynamical model
    replay_buffer.extend(data)
    states, actions = build_pendulum_training_data(data)

    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(
        num_tasks=states.shape[1]
    )
    model = DynamicalModel(
        states,
        actions,
        likelihood,
        position_memory=position_memory,
        control_memory=control_memory,
    )

    # Find optimal model hyperparameters
    fit(model, likelihood, print_loss = False)

In [64]:
from torch_pilco.policy_learning.rollout import policy_rollout
from torch_pilco.rewards import pendulum_cost

In [54]:
def pendulum_cost(
    states: torch.Tensor,
    actions: torch.Tensor,
) -> float:
    """
    Replicated Cost function from gymnasium:
        -(theta**2 + 0.1*theta_dt**2 + 0.001*torque**2)
    but we minimize it, so we return the negation.
    """

    x = states[0]
    y = states[1]
    angle_velocity = states[2]
    torque = actions[0]
    theta = torch.atan2(y, x)

    return (
        torch.square(theta) +
        0.1*torch.square(angle_velocity) +
        0.001*torch.square(torque)
    )

In [55]:
states.shape, actions.shape, torch.vmap(pendulum_cost, in_dims=(0,0))(states,actions).shape

(torch.Size([100, 3]), torch.Size([100, 1]), torch.Size([100]))

### Rollout
To do rollout, we:
1. sample a random draw from our current replay buffer
2. convert the replay buffer sample to a model input
3. sample from the dynamical model `num_particles` times
4. updates states
5. generate actions for each particle
   1. initially this will be random
   2. after the first iteration, will use the learnt policy
6. update running actions for each particle
7. convert the running state/action to a model input for each particle
8. sample from they dynamical model `1` time for each particle
9. repeat steps 4-7 until number of rollout steps is exausted

In [56]:
policy = RandomExploration(1,1,True,2.0)

In [57]:
# Generate an initial condition for rollout
# 1. sample a random draw from current replay buffer
replay_buffer_sample = replay_buffer.sample(queue_size)

In [58]:
# 2. convert the replay buffer sample to model input
initial_states, initial_actions = build_pendulum_training_data(replay_buffer_sample)
model_input = data_to_gp_input(initial_states, initial_actions, control_memory, position_memory)

In [59]:
# 3. sample from the dynamical model `num_particles` times
# Create the posterior:
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    posterior = model(model_input)
next_states = posterior.sample(sample_shape=torch.Size([num_particles]))
# Now inflate running_states (and running_actions) for each particle
running_states = torch.tile(initial_states,(num_particles,1,1))
running_actions = torch.tile(initial_actions,(num_particles,1,1))

In [60]:
# Update running_states
running_states = update_states(next_states, running_states)

In [63]:
policy_rollout(policy,running_states,running_actions,model,pendulum_cost)

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [20]:
# 4. Generate actions for each particle
action_inputs = torch.vmap(data_to_policy_input_closure, in_dims=0)(running_states)
next_actions = torch.vmap(policy, in_dims=0)(action_inputs)

In [21]:
running_actions = update_actions(next_actions, running_actions)

In [22]:
model_input = vmap(data_to_gp_input_closure,in_dims=(0,0))(running_states,running_actions)

In [23]:
model_input.shape

torch.Size([400, 1, 11])

### Should vmap the following using ensembling
like [here](https://docs.pytorch.org/tutorials/intermediate/ensembling.html)

In [24]:
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    posteriors = [model(model_input[i,:,:]) for i in range(num_particles)]

In [32]:
next_samples = [posteriors[i].sample() for i in range(num_particles)]

In [33]:
next_states = torch.cat(next_samples).unsqueeze(1)

In [34]:
next_states.shape

torch.Size([400, 1, 3])

In [35]:
# Update running_states
running_states = update_states(next_states, running_states)

In [36]:
# 4. Generate actions for each particle
action_inputs = torch.vmap(data_to_policy_input_closure, in_dims=0)(running_states)
next_actions = torch.vmap(policy, in_dims=0)(action_inputs)

In [37]:
running_actions = update_actions(next_actions, running_actions)

In [38]:
model_input = vmap(data_to_gp_input_closure,in_dims=(0,0))(running_states,running_actions)

In [39]:
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    posteriors = [model(model_input[i,:,:]) for i in range(num_particles)]

In [40]:
next_samples = [posteriors[i].sample() for i in range(num_particles)]

In [41]:
next_states = torch.cat(next_samples).unsqueeze(1)

In [42]:
next_states.shape

torch.Size([400, 1, 3])

In [43]:
type(model)

torch_pilco.model_learning.dynamical_models.DynamicalModel